# 三、特徵工程
<hr style="border:2px solid gray">

我們可以從 prior 的訂單中挖掘出顧客先前的消費行為和習慣，藉此提取出有用的特徵來訓練我們的模型，最後使用訓練好的模型來預測顧客重複購買的產品。

In [1]:
# 載入所需套件
import gc
import pandas as pd
from sklearn.decomposition import NMF
from sklearn.preprocessing import normalize

In [2]:
# 讀取數據
order_products_prior_df = pd.read_csv('./datasets/order_products__prior.csv')
order_products_train_df = pd.read_csv('./datasets/order_products__train.csv')
orders_df = pd.read_csv('./datasets/orders.csv')
products_df = pd.read_csv('./datasets/products.csv')
aisles_df = pd.read_csv('./datasets/aisles.csv')
departments_df = pd.read_csv('./datasets/departments.csv')

In [3]:
# 合併先前訂單和產品的數據
prd = orders_df.merge(order_products_prior_df, on='order_id', how='inner')
prd = prd.merge(products_df, on='product_id', how='left')
prd = prd.merge(aisles_df, on='aisle_id', how='left')
prd = prd.merge(departments_df, on='department_id', how='left')
print("Shape of prd :", prd.shape)
prd.head()

Shape of prd : (32434489, 15)


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department
0,2539329,1,prior,1,2,8,NaN,196,1,0,Soda,77,7,soft drinks,beverages
1,2539329,1,prior,1,2,8,NaN,14084,2,0,Organic Unsweetened Vanilla Almond Milk,91,16,soy lactosefree,dairy eggs
2,2539329,1,prior,1,2,8,NaN,12427,3,0,Original Beef Jerky,23,19,popcorn jerky,snacks
3,2539329,1,prior,1,2,8,NaN,26088,4,0,Aged White Cheddar Popcorn,23,19,popcorn jerky,snacks
4,2539329,1,prior,1,2,8,NaN,26405,5,0,XL Pick-A-Size Paper Towel Rolls,54,17,paper goods,household


In [4]:
# 計算在所有訂單中，有多少比例是唯一的 user_id 和 product_id 組合
prd.groupby(["user_id","product_id"]).size().shape[0]/prd.shape[0]

0.41030253320778387

# 3-1. 產品特徵
<hr style="border:2px solid gray">

創建基於產品特徵的 DataFrame。

最後會將此 DataFrame 會與其他的 DataFrame 合併成訓練集。

特徵：

1. `product_reorder_rate`：產品被重複購買的頻率
2. `product_avg_pos_incart`：產品在購物車中的平均順位
3. `product_reduced_feat_1` : NMF 輸出的第 1 列
4. `product_reduced_feat_2` : NMF 輸出的第 2 列
5. `product_reduced_feat_3` : NMF 輸出的第 3 列
6. `product_aisle_reorder_rate` : 產品所在通道的重複訂購頻率
7. `product_department_reorder_rate` : 產品所在部門的重複訂購頻率

### 3-1-1. 產品被重複購買的頻率(product_reorder_rate)

In [5]:
products = prd.groupby('product_id')['reordered'].mean().reset_index(name='product_reorder_ratio')
products.head()

,product_id,product_reorder_ratio
0,1,0.613391
1,2,0.133333
2,3,0.732852
3,4,0.446809
4,5,0.600000


### 3-1-2. 產品在購物車中的平均順位(product_avg_pos_incart)

In [6]:
product_var = prd.groupby('product_id')['add_to_cart_order'].mean().reset_index(name='product_avg_pos_incart')
products = products.merge(product_var, on='product_id', how='left')
products.head()

,product_id,product_reorder_ratio,product_avg_pos_incart
0,1,0.613391,5.801836
1,2,0.133333,9.888889
2,3,0.732852,6.415162
3,4,0.446809,9.507599
4,5,0.600000,6.466667


### 3-1-3. 產品是否屬於特定類別(product_reduced_feat_1-3)

基於產品是否屬於以下類別：

- organic（有機的）
- isYogurt（是否為酸奶 - aisle）
- isProduce（是否為農產品 - department）
- isFrozen（是否為冷凍食品 - department）
- isdairy（是否為乳製品 - department）
- isbreakfast（是否為早餐食品 - department）
- issnack（是否為零食 - department）
- isbeverage（是否為飲料 - department）

使用非負矩陣分解（NMF）將這些值降維為三列，藉此減少其稀疏性。

參考作法：https://www.kaggle.com/code/themissingsock/matrix-decomposition-with-buyer-data/notebook

In [7]:
products_df['organic'] = products_df['product_name'].apply(lambda x: 'organic' in x.lower()).astype(int)
products_df['isYogurt'] = products_df['aisle_id'].apply(lambda x: x==120).astype(int)
products_df['isProduce'] = products_df['department_id'].apply(lambda x: x==4).astype(int)
products_df['isFrozen'] = products_df['department_id'].apply(lambda x: x==1).astype(int)
products_df['isdairy'] = products_df['department_id'].apply(lambda x: x==16).astype(int)
products_df['isbreakfast'] = products_df['department_id'].apply(lambda x: x==14).astype(int)
products_df['issnack'] = products_df['department_id'].apply(lambda x: x==19).astype(int)
products_df['isbeverage'] = products_df['department_id'].apply(lambda x: x==7).astype(int)

In [8]:
product_var = products_df[['organic', 'isYogurt', 'isProduce', 'isFrozen', 'isdairy', 'isbreakfast', 'issnack', 'isbeverage']]
nmf = NMF(n_components=3, random_state=123)
model = nmf.fit(product_var)
W = model.transform(product_var)
product_var = pd.DataFrame(normalize(W))
product_var.columns = ['product_reduced_feat_1', 'product_reduced_feat_2','product_reduced_feat_3']

products['product_reduced_feat_1'] = product_var['product_reduced_feat_1']
products['product_reduced_feat_2'] = product_var['product_reduced_feat_2']
products['product_reduced_feat_3'] = product_var['product_reduced_feat_3']

products_df.drop(['organic', 'isYogurt', 'isProduce', 'isFrozen', 'isdairy', 'isbreakfast', 'issnack', 'isbeverage'], axis=1, inplace=True)

products.head()

,product_id,product_reorder_ratio,product_avg_pos_incart,product_reduced_feat_1,product_reduced_feat_2,product_reduced_feat_3
0,1,0.613391,5.801836,1.0,0.0,0.0
1,2,0.133333,9.888889,0.0,0.0,0.0
2,3,0.732852,6.415162,0.0,0.0,1.0
3,4,0.446809,9.507599,0.0,1.0,0.0
4,5,0.600000,6.466667,0.0,0.0,0.0


### 3-1-4. 產品所在通道的重複訂購頻率(product_aisle_reorder_rate)

In [9]:
products = products.merge(prd[['product_id', 'aisle_id']].drop_duplicates(), on='product_id', how='left')

product_var = prd.groupby('aisle_id')['reordered'].mean().reset_index(name='product_aisle_reorder_rate')

products = products.merge(product_var, on='aisle_id', how='left')

products.drop(['aisle_id'], axis=1, inplace=True)

products.head()

,product_id,product_reorder_ratio,product_avg_pos_incart,product_reduced_feat_1,product_reduced_feat_2,product_reduced_feat_3,product_aisle_reorder_rate
0,1,0.613391,5.801836,1.0,0.0,0.0,0.548698
1,2,0.133333,9.888889,0.0,0.0,0.0,0.152391
2,3,0.732852,6.415162,0.0,0.0,1.0,0.527615
3,4,0.446809,9.507599,0.0,1.0,0.0,0.556655
4,5,0.600000,6.466667,0.0,0.0,0.0,0.280627


### 3-1-5. 產品所在部門的重複訂購頻率(product_department_reorder_rate)

In [10]:
products = products.merge(prd[['product_id', 'department_id']].drop_duplicates(), on='product_id', how='left')

product_var = prd.groupby('department_id')['reordered'].mean().reset_index(name='product_department_reorder_rate')

products = products.merge(product_var, on='department_id', how='left')

products.drop(['department_id'], axis=1, inplace=True)

products.head()

,product_id,product_reorder_ratio,product_avg_pos_incart,product_reduced_feat_1,product_reduced_feat_2,product_reduced_feat_3,product_aisle_reorder_rate,product_department_reorder_rate
0,1,0.613391,5.801836,1.0,0.0,0.0,0.548698,0.574180
1,2,0.133333,9.888889,0.0,0.0,0.0,0.152391,0.346721
2,3,0.732852,6.415162,0.0,0.0,1.0,0.527615,0.653460
3,4,0.446809,9.507599,0.0,1.0,0.0,0.556655,0.541885
4,5,0.600000,6.466667,0.0,0.0,0.0,0.280627,0.346721


In [11]:
# 清除暫存
del product_var
gc.collect()

0

# 3-2. 用戶特徵
<hr style="border:2px solid gray">

創建基於用戶特徵的 DataFrame。

最後會將此 DataFrame 會與其他的 DataFrame 合併成訓練集。

特徵：

1. `user_reorder_rate`：用戶的平均重複訂購率
2. `user_unique_products`：用戶訂購的獨特產品數量
3. `user_total_products` : 用戶訂購的總產品數量
4. `user_avg_cart_size` : 用戶每次訂購的平均購物車大小
5. `user_avg_days_between_orders` : 用戶訂購平均的間隔天數
6. `user_reordered_products_ratio` : 用戶有重新訂購過產品的比例

### 3-2-1. 用戶的平均重複訂購率(user_reorder_rate)

In [12]:
users = prd.groupby('user_id')['reordered'].mean().reset_index(name='user_reorder_rate')
users.head()

,user_id,user_reorder_rate
0,1,0.694915
1,2,0.476923
2,3,0.625000
3,4,0.055556
4,5,0.378378


### 3-2-2. 用戶訂購的獨特產品數量(user_unique_products)

In [13]:
user_var = prd.groupby('user_id')['product_id'].nunique().reset_index(name='user_unique_products')

users = users.merge(user_var[['user_id', 'user_unique_products']], on='user_id', how='left')

users.head()

,user_id,user_reorder_rate,user_unique_products
0,1,0.694915,18
1,2,0.476923,102
2,3,0.625000,33
3,4,0.055556,17
4,5,0.378378,23


### 3-2-3. 用戶訂購的總產品數量(user_total_products)

In [14]:
user_var = prd.groupby('user_id').size().reset_index(name='user_total_products')

users = users.merge(user_var, on='user_id', how='left')

users.head()

,user_id,user_reorder_rate,user_unique_products,user_total_products
0,1,0.694915,18,59
1,2,0.476923,102,195
2,3,0.625000,33,88
3,4,0.055556,17,18
4,5,0.378378,23,37


### 3-2-4. 用戶每次訂購的平均購物車大小(user_avg_cart_size)

In [15]:
user_var = prd.groupby('user_id')['order_number'].max().reset_index(name='user_total_orders')

user_var['user_avg_cart_size'] = users['user_total_products'] / user_var['user_total_orders']
users = users.merge(user_var[['user_id', 'user_avg_cart_size']], on='user_id', how='left')

users.head()

,user_id,user_reorder_rate,user_unique_products,user_total_products,user_avg_cart_size
0,1,0.694915,18,59,5.900000
1,2,0.476923,102,195,13.928571
2,3,0.625000,33,88,7.333333
3,4,0.055556,17,18,3.600000
4,5,0.378378,23,37,9.250000


### 3-2-5. 用戶訂購平均的間隔天數(user_avg_days_between_orders)

需特別注意的是，新用戶第一次購買的間隔天數會是NaN，因此此特徵是扣除第一次購買的平均。

In [16]:
user_var = orders_df[orders_df['eval_set'] == 'prior'].groupby("user_id")['days_since_prior_order'].mean().reset_index(name='user_avg_days_between_orders')

users = users.merge(user_var, on='user_id', how='left')

users.head()

,user_id,user_reorder_rate,user_unique_products,user_total_products,user_avg_cart_size,user_avg_days_between_orders
0,1,0.694915,18,59,5.900000,19.555556
1,2,0.476923,102,195,13.928571,15.230769
2,3,0.625000,33,88,7.333333,12.090909
3,4,0.055556,17,18,3.600000,13.750000
4,5,0.378378,23,37,9.250000,13.333333


### 3-2-6. 用戶有重新訂購過產品的比例(user_reordered_products_ratio)

In [17]:
# 先計算出用戶有重複購買過的產品有哪些，接著再除以用戶購買過的獨特產品數量
user_var = prd[prd['reordered'] == 1].groupby('user_id')['product_id'].nunique().reset_index(name='user_reordered_products')

user_var['user_reordered_products_ratio'] = user_var['user_reordered_products'] / users['user_unique_products']
users = users.merge(user_var[['user_id', 'user_reordered_products_ratio']], on='user_id', how='left')

# 注意：有些用戶從來沒有重複購買過，因此合併後匯市Nan，記得將其比例補 0
users.fillna(value = 0, inplace = True)

users.head()

,user_id,user_reorder_rate,user_unique_products,user_total_products,user_avg_cart_size,user_avg_days_between_orders,user_reordered_products_ratio
0,1,0.694915,18,59,5.900000,19.555556,0.555556
1,2,0.476923,102,195,13.928571,15.230769,0.362745
2,3,0.625000,33,88,7.333333,12.090909,0.575758
3,4,0.055556,17,18,3.600000,13.750000,0.058824
4,5,0.378378,23,37,9.250000,13.333333,0.347826


In [18]:
# 清除暫存
del user_var
gc.collect()

10

# 3-3. 用戶-產品特徵
<hr style="border:2px solid gray">

創建基於 用戶-產品 交互特徵的 DataFrame。

最後會將此 DataFrame 會與其他的 DataFrame 合併成訓練集。

特徵：

1. `uxp_order_rate`：用戶訂購該產品的頻率
2. `uxp_reorder_rate`：用戶重新訂購該產品的頻率
3. `uxp_avg_position` : 用戶下單時該產品在購物車中的平均順位
4. `uxp_orders_since_last` : 用戶上次訂購該產品以來下的訂單數
5. `uxp_max_streak` : 用戶連續不間斷購買該產品最大的訂單數

### 3-3-1. 用戶訂購該產品的頻率(uxp_order_rate)

In [19]:
# 先計算用戶購買該產品的次數，再除以用戶的總訂單數
uxp = prd.groupby(['user_id', 'product_id'])['order_id'].count().reset_index(name='uxp_total_count')
user_var = prd.groupby('user_id')['order_id'].nunique().reset_index(name='user_total_order_count')

uxp = uxp.merge(user_var, on='user_id', how='left')
uxp['uxp_order_rate'] = uxp['uxp_total_count'] / uxp['user_total_order_count']

uxp.drop(['uxp_total_count', 'user_total_order_count'], axis=1, inplace=True)

uxp.head()

,user_id,product_id,uxp_order_rate
0,1,196,1.0
1,1,10258,0.9
2,1,10326,0.1
3,1,12427,1.0
4,1,13032,0.3


### 3-3-2. 用戶重新訂購該產品的頻率(uxp_reorder_rate)

In [20]:
# 用戶 "首次購買每個產品的訂單號"
uxp_var = prd.groupby(['user_id', 'product_id'])['order_number'].min().reset_index(name='uxp_first_order_number')

# 用戶的總訂單數
user_var = prd.groupby('user_id')['order_id'].nunique().reset_index(name='user_total_order_count')

uxp_var = uxp_var.merge(user_var, on='user_id', how='left')

# 用戶購買該產品的總次數
uxp_var2 = prd.groupby(['user_id', 'product_id'])['order_id'].count().reset_index(name='uxp_total_count')

# 後續用戶購買該產品的總次數(扣掉首次購買的次數) / 後續訂單數(包含首次購買)
uxp_var['uxp_order_reorder_ratio'] = (uxp_var2['uxp_total_count'] - 1) / (uxp_var['user_total_order_count'] - uxp_var['uxp_first_order_number'] + 1)

uxp = uxp.merge(uxp_var[['user_id', 'product_id', 'uxp_order_reorder_ratio']], on=['user_id', 'product_id'], how='left')

del user_var, uxp_var, uxp_var2
gc.collect()

uxp.head()

,user_id,product_id,uxp_order_rate,uxp_order_reorder_ratio
0,1,196,1.0,0.900000
1,1,10258,0.9,0.888889
2,1,10326,0.1,0.000000
3,1,12427,1.0,0.900000
4,1,13032,0.3,0.222222


### 3-3-3. 用戶下單時該產品在購物車中的平均順位(uxp_avg_position)

In [21]:
uxp_var = prd.groupby(['user_id', 'product_id'])['add_to_cart_order'].mean().reset_index(name='uxp_avg_position')

uxp = uxp.merge(uxp_var, on=['user_id', 'product_id'], how='left')

del uxp_var
gc.collect()

uxp.head()

,user_id,product_id,uxp_order_rate,uxp_order_reorder_ratio,uxp_avg_position
0,1,196,1.0,0.900000,1.400000
1,1,10258,0.9,0.888889,3.333333
2,1,10326,0.1,0.000000,5.000000
3,1,12427,1.0,0.900000,3.300000
4,1,13032,0.3,0.222222,6.333333


### 3-3-4. 用戶上次訂購該產品以來下的訂單數(uxp_orders_since_last)

In [22]:
user_var = prd.groupby('user_id')['order_id'].nunique().reset_index(name='user_total_order_count')


uxp_var = prd.groupby(['user_id', 'product_id'])['order_number'].max().reset_index(name='product_last_order_number')

uxp_var = uxp_var.merge(user_var, on='user_id', how='left')
uxp_var['uxp_orders_since_last'] = uxp_var['user_total_order_count'] - uxp_var['product_last_order_number']

uxp = uxp.merge(uxp_var[['user_id', 'product_id', 'uxp_orders_since_last']], on=['user_id', 'product_id'], how='left')

del user_var, uxp_var
gc.collect()

uxp.head()

,user_id,product_id,uxp_order_rate,uxp_order_reorder_ratio,uxp_avg_position,uxp_orders_since_last
0,1,196,1.0,0.900000,1.400000,0
1,1,10258,0.9,0.888889,3.333333,0
2,1,10326,0.1,0.000000,5.000000,5
3,1,12427,1.0,0.900000,3.300000,0
4,1,13032,0.3,0.222222,6.333333,0


### 3.3.5. 用戶連續不間斷購買該產品最大的訂單數(uxp_max_streak)

In [23]:
def max_streak(row):
    """
    計算用戶連續訂購該產品的最大訂單數
    inp: pandas row
    output: max orders
    """
    _max = 0 # 用來儲存最大的連續重新訂購次數
    _sum = -1 # 用來計算當前的連續重新訂購次數，-1是因為一定有第一次購買的紀錄
    for i in row:
        if i == 1: # 如果訂單中訂購了該產品
            _sum += 1 # 當前連續訂購次數增加 1
        else:
            if _sum > _max: # 如果當前連續訂購次數大於已知的最大連續訂購次數
                _max = _sum # 更新最大連續訂購次數
            _sum = -1 # 重置當前連續訂購次數
    if _sum > _max:  # 最後一個訂單如果是連續的也要考慮
        _max = _sum
    return _max

In [24]:
# 先標記第一次購買
uxp_var = prd.groupby(["user_id", "product_id"])["order_number"].min().reset_index(name="uxp_first_order_number")
# 合併首次訂購訂單編號到原始數據
uxp_var = prd.merge(uxp_var, on=["user_id", "product_id"], how="left")
# 首次訂購訂單的位置上將 reordered 設為 1
uxp_var.loc[uxp_var['order_number'] == uxp_var['uxp_first_order_number'], 'reordered'] = 1
# 計算每個 user_id 和 product_id 的重新訂購列表
uxp_var = uxp_var.groupby(["user_id", "product_id"])["reordered"].agg(list).reset_index(name="order_record")
# 計算最大連續重新訂購次數
uxp_var["uxp_max_streak"] = uxp_var["order_record"].apply(max_streak)

uxp = uxp.merge(uxp_var[['user_id', 'product_id', 'uxp_max_streak']], on=['user_id', 'product_id'], how='left')

del uxp_var
gc.collect()

uxp.head()

,user_id,product_id,uxp_order_rate,uxp_order_reorder_ratio,uxp_avg_position,uxp_orders_since_last,uxp_max_streak
0,1,196,1.0,0.900000,1.400000,0,9
1,1,10258,0.9,0.888889,3.333333,0,8
2,1,10326,0.1,0.000000,5.000000,5,0
3,1,12427,1.0,0.900000,3.300000,0,9
4,1,13032,0.3,0.222222,6.333333,0,2


# 3-4. 結合 用戶-產品 的產品特徵
<hr style="border:2px solid gray">

創建基於 用戶-產品 的產品特徵 DataFrame。

特徵：

1. `uxp_product_unique_customers`：產品的獨立用戶數
2. `uxp_product_one_shot_ratio`：產品只有被單一次購買的比例

### 3-4-1. 產品的獨立用戶數(uxp_product_unique_customers)

In [25]:
product_var = uxp.groupby('product_id')['user_id'].count().reset_index(name='uxp_product_unique_customers')

products = products.merge(product_var, on='product_id', how='left')

del product_var
gc.collect()

products.head()

,product_id,product_reorder_ratio,product_avg_pos_incart,product_reduced_feat_1,product_reduced_feat_2,product_reduced_feat_3,product_aisle_reorder_rate,product_department_reorder_rate,uxp_product_unique_customers
0,1,0.613391,5.801836,1.0,0.0,0.0,0.548698,0.574180,716
1,2,0.133333,9.888889,0.0,0.0,0.0,0.152391,0.346721,78
2,3,0.732852,6.415162,0.0,0.0,1.0,0.527615,0.653460,74
3,4,0.446809,9.507599,0.0,1.0,0.0,0.556655,0.541885,182
4,5,0.600000,6.466667,0.0,0.0,0.0,0.280627,0.346721,6


### 3-4-2. 產品只有被用戶單一次購買的比例(uxp_product_one_shot_ratio)

In [26]:
uxp_var = prd.groupby(['user_id', 'product_id'])['order_id'].count().reset_index(name='uxp_total_count')

product_var = uxp_var[uxp_var.uxp_total_count == 1].groupby('product_id')['uxp_total_count'].count().reset_index(name='uxp_product_one_shot')

products = products.merge(product_var, on='product_id', how='left')
products['uxp_product_one_shot_ratio'] = products['uxp_product_one_shot'] / products['uxp_product_unique_customers']
products.drop(['uxp_product_one_shot'], axis=1, inplace=True)

del uxp_var, product_var
gc.collect()

products.head()

,product_id,product_reorder_ratio,product_avg_pos_incart,product_reduced_feat_1,product_reduced_feat_2,product_reduced_feat_3,product_aisle_reorder_rate,product_department_reorder_rate,uxp_product_unique_customers,uxp_product_one_shot_ratio
0,1,0.613391,5.801836,1.0,0.0,0.0,0.548698,0.574180,716,0.614525
1,2,0.133333,9.888889,0.0,0.0,0.0,0.152391,0.346721,78,0.897436
2,3,0.732852,6.415162,0.0,0.0,1.0,0.527615,0.653460,74,0.513514
3,4,0.446809,9.507599,0.0,1.0,0.0,0.556655,0.541885,182,0.648352
4,5,0.600000,6.466667,0.0,0.0,0.0,0.280627,0.346721,6,0.333333


# 3-5. 其餘特徵
<hr style="border:2px solid gray">

特徵：

1. `misc_hour_order_rate`：產品每小時的訂購率
2. `misc_day_order_rate`：產品每星期幾的訂購率
3. `misc_days_since_prior_order_order_rate`：產品間隔天數的訂購率
4. `misc_user_days_since_prior_order_order_rate`：用戶間隔天數的訂購率
5. `misc_uxp_days_since_prior_order_order_rate`：用戶對於該產品間隔天數的訂購率

### 3-5-1. 產品每小時的訂購率(misc_hour_reorder_rate)

In [27]:
hour_order_rate = (prd.groupby(['product_id', 'order_hour_of_day']).size() / prd.groupby(["product_id"]).size()).reset_index(name="misc_hour_order_rate")
hour_order_rate.head()

,product_id,order_hour_of_day,misc_hour_order_rate
0,1,0,0.006479
1,1,1,0.005940
2,1,2,0.004860
3,1,3,0.002700
4,1,4,0.002160


### 3-5-2. 產品每星期幾的訂購率(misc_day_order_rate)

In [28]:
day_order_rate = (prd.groupby(['product_id', 'order_dow']).size() / prd.groupby(["product_id"]).size()).reset_index(name="misc_day_order_rate")
day_order_rate.head()

,product_id,order_dow,misc_day_order_rate
0,1,0,0.108531
1,1,1,0.215983
2,1,2,0.146328
3,1,3,0.141469
4,1,4,0.159287


### 3-5-3. 產品間隔天數的訂購率(misc_product_days_since_prior_order_order_rate)

In [29]:
product_days_since_prior_order_order_rate = (prd.groupby(['product_id', 'days_since_prior_order']).size() / prd.groupby(["product_id"]).size()).reset_index(name="misc_product_days_since_prior_order_order_rate")
product_days_since_prior_order_order_rate.head()

,product_id,days_since_prior_order,misc_product_days_since_prior_order_order_rate
0,1,0.0,0.049136
1,1,1.0,0.051836
2,1,2.0,0.048596
3,1,3.0,0.055616
4,1,4.0,0.041037


### 3-5-4. 用戶間隔天數的訂購率(misc_user_days_since_prior_order_order_rate)

In [30]:
user_days_since_prior_order_order_rate = (prd.groupby(['user_id', 'days_since_prior_order']).size() / prd.groupby(["user_id"]).size()).reset_index(name="misc_user_days_since_prior_order_order_rate")
user_days_since_prior_order_order_rate.head()

,user_id,days_since_prior_order,misc_user_days_since_prior_order_order_rate
0,1,0.0,0.101695
1,1,14.0,0.101695
2,1,15.0,0.101695
3,1,19.0,0.067797
4,1,20.0,0.084746


### 3-5-5. 用戶對於該產品間隔天數的訂購率(misc_uxp_days_since_prior_order_order_rate)

In [31]:
uxp_days_since_prior_order_order_rate = (prd.groupby(['user_id', 'product_id', 'days_since_prior_order']).size() / prd.groupby(["user_id", 'product_id']).size()).reset_index(name="misc_uxp_days_since_prior_order_order_rate")
uxp_days_since_prior_order_order_rate.head()

,user_id,product_id,days_since_prior_order,misc_uxp_days_since_prior_order_order_rate
0,1,196,0.0,0.1
1,1,196,14.0,0.1
2,1,196,15.0,0.1
3,1,196,19.0,0.1
4,1,196,20.0,0.1


### 3-5-6. 將時間轉換為類別變數

In [32]:
def hour_to_categorical(time):
    if 6 <= time < 12:  # 6 AM 到 11 AM
        return 0
    elif 12 <= time < 17:  # 12 PM 到 4 PM
        return 1
    elif 17 <= time < 21:  # 5 PM 到 8 PM
        return 2
    else:  # 9 PM 到 5 AM
        return 3

# 3-6. 創建訓練集、測試集
<hr style="border:2px solid gray">

### 3-6-1. 合併 users, products, uxp 的 DataFrame

In [33]:
data = uxp.merge(users, on='user_id', how='left')
data = data.merge(products, on='product_id', how='left')
data

,user_id,product_id,uxp_order_rate,uxp_order_reorder_ratio,uxp_avg_position,uxp_orders_since_last,uxp_max_streak,user_reorder_rate,user_unique_products,user_total_products,...,user_reordered_products_ratio,product_reorder_ratio,product_avg_pos_incart,product_reduced_feat_1,product_reduced_feat_2,product_reduced_feat_3,product_aisle_reorder_rate,product_department_reorder_rate,uxp_product_unique_customers,uxp_product_one_shot_ratio
0,1,196,1.000000,0.900000,1.400000,0,9,0.694915,18,59,...,0.555556,0.776480,3.721774,0.000000,0.000000,1.000000,0.638832,0.653460,8000,0.417500
1,1,10258,0.900000,0.888889,3.333333,0,8,0.694915,18,59,...,0.555556,0.713772,4.277492,0.000000,0.000000,0.000000,0.519170,0.574180,557,0.447038
2,1,10326,0.100000,0.000000,5.000000,5,0,0.694915,18,59,...,0.555556,0.652009,4.191097,0.000000,0.000000,0.000000,0.718104,0.649913,1923,0.478419
3,1,12427,1.000000,0.900000,3.300000,0,9,0.694915,18,59,...,0.555556,0.740735,4.760037,0.000000,0.000000,0.000000,0.591986,0.574180,1679,0.470518
4,1,13032,0.300000,0.222222,6.333333,0,2,0.694915,18,59,...,0.555556,0.657158,5.622767,0.000000,0.000000,0.000000,0.571584,0.560922,1286,0.520218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13307948,206209,43961,0.230769,0.200000,8.000000,1,2,0.472868,68,129,...,0.263736,0.630583,9.194723,0.000000,0.000000,0.000000,0.638514,0.649913,20455,0.485358
13307949,206209,44325,0.076923,0.000000,8.000000,6,0,0.472868,68,129,...,0.263736,0.401148,10.109900,0.000000,1.000000,0.000000,0.453062,0.461076,2087,0.710110
13307950,206209,48370,0.076923,0.000000,8.000000,2,0,0.472868,68,129,...,0.263736,0.699288,8.344942,0.000000,0.000000,0.000000,0.528005,0.402178,1183,0.407439
13307951,206209,48697,0.076923,0.000000,6.000000,6,0,0.472868,68,129,...,0.263736,0.357661,8.763058,0.002059,0.999984,0.005347,0.350843,0.346721,6284,0.739338


In [34]:
# 新增 order_dow, order_hour_of_day, days_since_prior_order, eval_set 和 order_id 到 data
order_future = orders_df.loc[((orders_df.eval_set == 'train') | (orders_df.eval_set == 'test')), ['user_id', 'order_dow', 'order_hour_of_day', 'days_since_prior_order', 'eval_set', 'order_id']]
order_future.head()

,user_id,order_dow,order_hour_of_day,days_since_prior_order,eval_set,order_id
10,1,4,8,14.0,train,1187899
25,2,1,11,30.0,train,1492625
38,3,5,15,11.0,test,2774568
44,4,3,12,30.0,test,329954
49,5,0,11,6.0,train,2196797


In [35]:
data = data.merge(order_future, on='user_id', how='left')
data.head()

,user_id,product_id,uxp_order_rate,uxp_order_reorder_ratio,uxp_avg_position,uxp_orders_since_last,uxp_max_streak,user_reorder_rate,user_unique_products,user_total_products,...,product_reduced_feat_3,product_aisle_reorder_rate,product_department_reorder_rate,uxp_product_unique_customers,uxp_product_one_shot_ratio,order_dow,order_hour_of_day,days_since_prior_order,eval_set,order_id
0,1,196,1.0,0.900000,1.400000,0,9,0.694915,18,59,...,1.0,0.638832,0.653460,8000,0.417500,4,8,14.0,train,1187899
1,1,10258,0.9,0.888889,3.333333,0,8,0.694915,18,59,...,0.0,0.519170,0.574180,557,0.447038,4,8,14.0,train,1187899
2,1,10326,0.1,0.000000,5.000000,5,0,0.694915,18,59,...,0.0,0.718104,0.649913,1923,0.478419,4,8,14.0,train,1187899
3,1,12427,1.0,0.900000,3.300000,0,9,0.694915,18,59,...,0.0,0.591986,0.574180,1679,0.470518,4,8,14.0,train,1187899
4,1,13032,0.3,0.222222,6.333333,0,2,0.694915,18,59,...,0.0,0.571584,0.560922,1286,0.520218,4,8,14.0,train,1187899


### 3-6-2. 新增其餘特徵

In [36]:
# 主要是因為要連同訓練集和測試集的資料一起轉換。
# 注意：訂單有些情況在先前的資料集中並未出現，合併後要記得補 0
data = data.merge(hour_order_rate, on=['product_id', 'order_hour_of_day'], how='left')
data = data.merge(day_order_rate, on=['product_id', 'order_dow'], how='left')
data = data.merge(product_days_since_prior_order_order_rate, on=['product_id', 'days_since_prior_order'], how='left')
data = data.merge(user_days_since_prior_order_order_rate, on=['user_id', 'days_since_prior_order'], how='left')
data = data.merge(uxp_days_since_prior_order_order_rate, on=['user_id', 'product_id', 'days_since_prior_order'], how='left')

data['order_hour_of_day'] = data['order_hour_of_day'].apply(hour_to_categorical)

data.fillna(value = 0, inplace = True)

### 3-6-3. 轉換資料型態

轉換資料型態可以顯著減少 DataFrame 的大小，從而減少記憶體的使用量。

較小的記憶體使用量可以提高數據處理的效能，尤其是在處理大型數據集時。

https://towardsdatascience.com/make-working-with-large-dataframes-easier-at-least-for-your-memory-6f52b5f4b5c4

In [37]:
def optimize_dataframe(df):
    import numpy as np
    # 定義不同資料型態的範圍
    int_types = {
        'int8': (-128, 127),
        'int16': (-32768, 32767),
        'int32': (-2147483648, 2147483647),
        'int64': (-9223372036854775808, 9223372036854775807)
    }
    uint_types = {
        'uint8': (0, 255),
        'uint16': (0, 65535),
        'uint32': (0, 4294967295),
        'uint64': (0, 18446744073709551615)
    }
    float_types = {
        'float16': (np.finfo(np.float16).min, np.finfo(np.float16).max),
        'float32': (np.finfo(np.float32).min, np.finfo(np.float32).max),
        'float64': (np.finfo(np.float64).min, np.finfo(np.float64).max)
    }

    for col in df.columns:
        col_type = df[col].dtype
        if pd.api.types.is_integer_dtype(col_type):
            col_min = df[col].min()
            col_max = df[col].max()
            for dtype, (min_val, max_val) in int_types.items():
                if col_min >= min_val and col_max <= max_val:
                    df[col] = df[col].astype(dtype)
                    break
        elif pd.api.types.is_unsigned_integer_dtype(col_type):
            col_min = df[col].min()
            col_max = df[col].max()
            for dtype, (min_val, max_val) in uint_types.items():
                if col_min >= min_val and col_max <= max_val:
                    df[col] = df[col].astype(dtype)
                    break
        elif pd.api.types.is_float_dtype(col_type):
            col_min = df[col].min()
            col_max = df[col].max()
            for dtype, (min_val, max_val) in float_types.items():
                if col_min >= min_val and col_max <= max_val:
                    df[col] = df[col].astype(dtype)
                    break
    return df

In [38]:
print(data.dtypes)
data = optimize_dataframe(data)
print('======================================')
print(data.dtypes)

user_id                                             int64
product_id                                          int64
uxp_order_rate                                    float64
uxp_order_reorder_ratio                           float64
uxp_avg_position                                  float64
uxp_orders_since_last                               int64
uxp_max_streak                                      int64
user_reorder_rate                                 float64
user_unique_products                                int64
user_total_products                                 int64
user_avg_cart_size                                float64
user_avg_days_between_orders                      float64
user_reordered_products_ratio                     float64
product_reorder_ratio                             float64
product_avg_pos_incart                            float64
product_reduced_feat_1                            float64
product_reduced_feat_2                            float64
product_reduce

### 3-6-4. 創建訓練集

In [39]:
data_train = data[data.eval_set == 'train'].copy()
data_train.head()

,user_id,product_id,uxp_order_rate,uxp_order_reorder_ratio,uxp_avg_position,uxp_orders_since_last,uxp_max_streak,user_reorder_rate,user_unique_products,user_total_products,...,order_dow,order_hour_of_day,days_since_prior_order,eval_set,order_id,misc_hour_order_rate,misc_day_order_rate,misc_product_days_since_prior_order_order_rate,misc_user_days_since_prior_order_order_rate,misc_uxp_days_since_prior_order_order_rate
0,1,196,1.000000,0.899902,1.400391,0,9,0.694824,18,59,...,4,0,14.0,train,1187899,0.059662,0.152588,0.038727,0.101685,0.099976
1,1,10258,0.899902,0.888672,3.333984,0,8,0.694824,18,59,...,4,0,14.0,train,1187899,0.062164,0.137695,0.041107,0.101685,0.111084
2,1,10326,0.099976,0.000000,5.000000,5,0,0.694824,18,59,...,4,0,14.0,train,1187899,0.066956,0.123962,0.035828,0.101685,0.000000
3,1,12427,1.000000,0.899902,3.300781,0,9,0.694824,18,59,...,4,0,14.0,train,1187899,0.065308,0.130127,0.041840,0.101685,0.099976
4,1,13032,0.300049,0.222168,6.332031,0,2,0.694824,18,59,...,4,0,14.0,train,1187899,0.061584,0.144531,0.032806,0.101685,0.000000


In [40]:
# 與 train 的 reordered 合併，要注意的是因為是用 left join 所以沒有出現在訂單中的產品 reordered 會是 NaN，最後要記得把 NaN 補 0
data_train = data_train.merge(order_products_train_df[['product_id', 'order_id', 'reordered']], on=['product_id', 'order_id'], how='left').fillna(0)
data_train.head()

,user_id,product_id,uxp_order_rate,uxp_order_reorder_ratio,uxp_avg_position,uxp_orders_since_last,uxp_max_streak,user_reorder_rate,user_unique_products,user_total_products,...,order_hour_of_day,days_since_prior_order,eval_set,order_id,misc_hour_order_rate,misc_day_order_rate,misc_product_days_since_prior_order_order_rate,misc_user_days_since_prior_order_order_rate,misc_uxp_days_since_prior_order_order_rate,reordered
0,1,196,1.000000,0.899902,1.400391,0,9,0.694824,18,59,...,0,14.0,train,1187899,0.059662,0.152588,0.038727,0.101685,0.099976,1.0
1,1,10258,0.899902,0.888672,3.333984,0,8,0.694824,18,59,...,0,14.0,train,1187899,0.062164,0.137695,0.041107,0.101685,0.111084,1.0
2,1,10326,0.099976,0.000000,5.000000,5,0,0.694824,18,59,...,0,14.0,train,1187899,0.066956,0.123962,0.035828,0.101685,0.000000,0.0
3,1,12427,1.000000,0.899902,3.300781,0,9,0.694824,18,59,...,0,14.0,train,1187899,0.065308,0.130127,0.041840,0.101685,0.099976,0.0
4,1,13032,0.300049,0.222168,6.332031,0,2,0.694824,18,59,...,0,14.0,train,1187899,0.061584,0.144531,0.032806,0.101685,0.000000,1.0


In [41]:
# 訓練集不需要用到 eval_set
data_train.drop(['eval_set'], axis=1, inplace=True)

### 3-6-5. 創建測試集

In [42]:
data_test = data[data.eval_set == 'test'].copy()
data_test.head()

,user_id,product_id,uxp_order_rate,uxp_order_reorder_ratio,uxp_avg_position,uxp_orders_since_last,uxp_max_streak,user_reorder_rate,user_unique_products,user_total_products,...,order_dow,order_hour_of_day,days_since_prior_order,eval_set,order_id,misc_hour_order_rate,misc_day_order_rate,misc_product_days_since_prior_order_order_rate,misc_user_days_since_prior_order_order_rate,misc_uxp_days_since_prior_order_order_rate
120,3,248,0.083313,0.000000,3.000000,10,0,0.625,33,88,...,5,1,11.0,test,2774568,0.088074,0.123230,0.021667,0.056824,0.0
121,3,1005,0.083313,0.000000,5.000000,2,0,0.625,33,88,...,5,1,11.0,test,2774568,0.051849,0.181396,0.015121,0.056824,0.0
122,3,1819,0.250000,0.222168,2.666016,5,2,0.625,33,88,...,5,1,11.0,test,2774568,0.071350,0.134033,0.025574,0.056824,0.0
123,3,7503,0.083313,0.000000,6.000000,9,0,0.625,33,88,...,5,1,11.0,test,2774568,0.073730,0.125244,0.024612,0.056824,0.0
124,3,8021,0.083313,0.000000,5.000000,10,0,0.625,33,88,...,5,1,11.0,test,2774568,0.078369,0.128174,0.026627,0.056824,0.0


In [43]:
# 測試集不需要用到 eval_set
data_test.drop(['eval_set'], axis=1, inplace=True)

# 3-7. 儲存數據集
<hr style="border:2px solid gray">

- CSV 格式：在讀取時，必須手動指定各列的資料型態以確保與儲存時一致。
- HDF5 格式：自動保留資料型態，在讀取時不需要手動指定資料型態。
- PKL 格式：是 Python 特有的格式，適合快速保存和讀取時使用。

In [44]:
# 儲存 train 和 test DataFrame 為 HDF5 格式
data_train.to_hdf('./datasets/train_test_data.h5', key='train', mode='w')
data_test.to_hdf('./datasets/train_test_data.h5', key='test', mode='a')
print("Data has been saved")

Data has been saved


In [45]:
hour_order_rate.to_pickle("./datasets/hour_order_rate.pkl")
day_order_rate.to_pickle("./datasets/day_order_rate.pkl")
product_days_since_prior_order_order_rate.to_pickle("./datasets/product_days_since_prior_order_order_rate.pkl")
user_days_since_prior_order_order_rate.to_pickle("./datasets/user_days_since_prior_order_order_rate.pkl")
uxp_days_since_prior_order_order_rate.to_pickle("./datasets/uxp_days_since_prior_order_order_rate.pkl")
print("Have been saved")

Have been saved
